In [1]:
 # Создаем spark сессию
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [2]:
# файл с оценками - user * item матрица
import os
import pyspark.sql.functions as sql_func

DATA_DIR = 'D:/Datasets/ml-latest-small/'
# Сначала посмотрим на общее распределение тегов
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    .drop('timestamp')
    .cache()
)

In [3]:
%%time
(
    ratings.alias("one")
    .join(ratings.alias("two"), "userId")
    # расстояние симметрично
    # поэтому считаем только одну сторону
    .where("one.movieId > two.movieId")
    .groupBy("one.movieId", "two.movieId")
    .agg(
        sql_func.sum(
            sql_func.col("one.rating") *
            sql_func.col("two.rating")
        ).alias("inner_product"),
        sql_func.count("userId").alias("watched_both")
    ).select(
        sql_func.col("one.movieId").alias("movieId1"),
        sql_func.col("two.movieId").alias("movieId2"),
        sql_func.col("watched_both"),
        sql_func.col("inner_product")
    )
    .write
    .mode("overwrite")
    .parquet("half_cooccurrences.parquet")
)

Wall time: 28.3 s


In [4]:
popularities = (
    ratings
    .groupBy("movieId")
    .agg(
        sql_func.sum(sql_func.pow(
            sql_func.col("rating"),
            2
        )).alias("sum_of_squares"),
        sql_func.count("userId").alias("watched_one")
    )
    .cache()
)

In [5]:
half_cooccurrences = (
    spark
    .read
    .parquet("half_cooccurrences.parquet")
)

In [6]:
# агрегаты с расчета расстояний - считается 15 минут
(
    half_cooccurrences
    .join(
        popularities.alias("pop1"),
        sql_func.col("pop1.movieId") == sql_func.col("movieId1")
    )
    .join(
        popularities.alias("pop2"),
        sql_func.col("pop2.movieId") == sql_func.col("movieId2")
    )
    .select(
        sql_func.col("movieId1"),
        sql_func.col("movieId2"),
        sql_func.col("pop1.watched_one").alias("watched1"),
        sql_func.col("pop2.watched_one").alias("watched2"),
        sql_func.col("pop1.sum_of_squares").alias("sum_of_squares1"),
        sql_func.col("pop2.sum_of_squares").alias("sum_of_squares2"),
        sql_func.col("inner_product"),
        sql_func.col("watched_both")
        
    )
    .write
    .mode("overwrite")
    .parquet("pre_distance_matrix.parquet")
)

In [7]:
pre_distance_matrix = (
    spark
    .read
    .parquet("pre_distance_matrix.parquet")
)

In [8]:
movies = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'movies.csv'),
        header=True,
        inferSchema=True
    )
    # если используется меньше памяти,
    # то здесь можно взять не все данные, а небольшую выборку
    # даже при fraction=.01 качественная картина не меняеся
    .select('movieId', 'title')
    .cache()
)

In [11]:
movieId = 4896
(
    pre_distance_matrix
    .where("movieId1 == {} OR movieId2 == {}".format(movieId, movieId))
    .selectExpr(
        """
        CASE
            WHEN movieId1 == {}
            THEN movieId2
            ELSE movieId1
        END movieId
        """.format(movieId),
#         """
#         ВАШ КОД ТУТ AS disctance
#         """
    )
#     .orderBy("distance")
    .limit(10)
    .join(movies, "movieId")
#     .orderBy("distance")
    .toPandas()
)

,movieId,title
0,2724,Runaway Bride (1999)
1,3268,Stop! Or My Mom Will Shoot (1992)
2,3477,Empire Records (1995)
3,3668,Romeo and Juliet (1968)
4,54881,"King of Kong, The (2007)"
5,86377,Louis C.K.: Shameless (2007)
6,100498,"Good Day to Die Hard, A (2013)"
7,115680,Time Lapse (2014)
8,117851,Penguins of Madagascar (2014)
9,182715,Annihilation (2018)
